<a href="https://colab.research.google.com/github/areeseb/demographicsofsound/blob/main/noise_pollution_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and Mount Drive**

In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 22.1 MB/s 
     |████████████████████████████████| 6.3 MB 35.9 MB/s 
     |████████████████████████████████| 15.4 MB 41.8 MB/s 


In [ ]:
!pip install libpysal

     |████████████████████████████████| 2.4 MB 34.1 MB/s 


In [ ]:
!pip install spglm

     |████████████████████████████████| 209 kB 29.6 MB/s 
  Created wheel for spglm: filename=spglm-1.0.8-py3-none-any.whl size=38806 sha256=da126576ced16f0b4e79cd5eed402ec26f909992cc734bcf0f0a156f430789b7
  Stored in directory: /root/.cache/pip/wheels/29/79/e9/453e3b0d858bed16e097c6e6bbd75e2b966bfcfb21b12790d0
Successfully built spglm


In [ ]:
!pip install mgwr

     |████████████████████████████████| 41 kB 609 kB/s 
  Created wheel for mgwr: filename=mgwr-2.1.2-py3-none-any.whl size=46386 sha256=e1812e8726e40953d44174cb12272d2b7d342ada7e5d74868c076035911a2509
  Stored in directory: /root/.cache/pip/wheels/e8/55/57/c1bf993ab0ab4b9885ac1fdf7f9f960397355cf889b77a3e4b
Successfully built mgwr


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import libpysal as ps
import numpy as np
import numpy as np
import libpysal as ps
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **GWR (T: local noise P: diversity index, pct_hisp_pop)**

In [ ]:
sound_gdf = gpd.read_file(r'/content/drive/My Drive/550/Final_Project/demographic_sounds_points_clean.geojson')

In [ ]:
sound_gdf.head()

,OID,avg_hh_size,pop_growth_rt,med_age,unemply_rate,hisp_pop,wht_pop,blk_pop,ai_na_pop,asn_pop,pi_pop,othr_pop,mtpl_pop,divst_indx,med_inc,avg_inc,local_noise,airport_noise,traffic_noise,sound_score,tot_pop_sum,pct_pop_growth_rt,pct_hisp_pop,pct_wht_pop,pct_blk_pop,pct_ai_na_pop,pct_asn_pop,pct_pi_pop,pct_othr_pop,pct_mtpl_pop,geometry
0,1,2.76,0.27,34.8,6.2,3110,341,1783,12,204,4,24,166,91.3,46092,60922,4.0,0.0,26.0,69.0,5644,0.000048,0.551028,0.060418,0.315911,0.002126,0.036145,0.000709,0.004252,0.029412,POINT (-118.35353 34.04261)
1,2,2.04,0.10,45.2,3.8,280,2952,71,4,168,0,21,123,35.0,140157,208204,0.0,0.0,26.0,72.0,3619,0.000028,0.077369,0.815695,0.019619,0.001105,0.046422,0.000000,0.005803,0.033987,POINT (-118.37353 34.09685)
2,477,2.04,0.10,45.2,3.8,280,2952,71,4,168,0,21,123,35.0,140157,208204,0.0,0.0,23.0,76.0,3619,0.000028,0.077369,0.815695,0.019619,0.001105,0.046422,0.000000,0.005803,0.033987,POINT (-118.37863 34.10206)
3,1030,2.04,0.10,45.2,3.8,280,2952,71,4,168,0,21,123,35.0,140157,208204,1.0,0.0,27.0,70.0,3619,0.000028,0.077369,0.815695,0.019619,0.001105,0.046422,0.000000,0.005803,0.033987,POINT (-118.36754 34.09751)
4,1233,2.04,0.10,45.2,3.8,280,2952,71,4,168,0,21,123,35.0,140157,208204,0.0,0.0,23.0,76.0,3619,0.000028,0.077369,0.815695,0.019619,0.001105,0.046422,0.000000,0.005803,0.033987,POINT (-118.37418 34.10288)


In [ ]:
#last little bit fo cleaning
mean_value = sound_gdf['pct_othr_pop'].mean()
sound_gdf['pct_othr_pop'].fillna(value=mean_value, inplace=True)
mean_value = sound_gdf['pct_hisp_pop'].mean()
sound_gdf['pct_hisp_pop'].fillna(value=mean_value, inplace=True)
mean_value = sound_gdf['pct_wht_pop'].mean()
sound_gdf['pct_wht_pop'].fillna(value=mean_value, inplace=True)

In [ ]:
#dataframe inputs
Y = sound_gdf['local_noise'].values.reshape((-1,1))

X = sound_gdf[['divst_indx','pct_hisp_pop']].values
u = sound_gdf['geometry'].x
v = sound_gdf['geometry'].y
coords = list(zip(u,v))

#scaling np imput
X = (X - X.mean(axis=0)) / X.std(axis=0)

Y = Y.reshape((-1,1))

Y = (Y - Y.mean(axis=0)) / Y.std(axis=0)

In [ ]:
#Calibrate GWR model
gwr_selector = Sel_BW(coords, Y, X)
gwr_bw = gwr_selector.search(verbose=True,bw_min=82)
print(gwr_bw)
gwr_results = GWR(coords, Y, X, gwr_bw).fit()

Bandwidth:  1006.0 , score:  6057.26
Bandwidth:  1576.0 , score:  6230.57
Bandwidth:  653.0 , score:  5850.61
Bandwidth:  435.0 , score:  5651.80
Bandwidth:  300.0 , score:  5503.65
Bandwidth:  217.0 , score:  5397.77
Bandwidth:  165.0 , score:  5312.58
Bandwidth:  134.0 , score:  5238.65
Bandwidth:  114.0 , score:  5168.32
Bandwidth:  102.0 , score:  5108.13
Bandwidth:  94.0 , score:  5062.20
Bandwidth:  90.0 , score:  5036.37
Bandwidth:  87.0 , score:  5014.53
Bandwidth:  85.0 , score:  5000.89
Bandwidth:  84.0 , score:  4993.44
Bandwidth:  83.0 , score:  4986.77
83.0


In [ ]:
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                2500
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                           2000.640
Log-likelihood:                                                   -3268.817
AIC:                                                               6543.634
AICc:                                                              6545.650
BIC:                                                             -17536.003
R2:                                                                   0.200
Adj. R2:                                                              0.199

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

# **General Multiple Linear Regression**

In [ ]:
Y = sound_gdf['local_noise']
X = sound_gdf[['divst_indx','pct_hisp_pop']]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)


In [ ]:
mlr = LinearRegression()
mlr.fit(x_train, y_train)

LinearRegression()

In [ ]:
print("Intercept: ", mlr.intercept_)
print("Coefficients:")
list(zip(X, mlr.coef_))

Intercept:  -0.7981962540080567
Coefficients:


[('divst_indx', 0.023382907861853312), ('pct_hisp_pop', 0.8436869480951132)]

In [ ]:
#Prediction of test set
y_pred_mlr= mlr.predict(x_test)
#Predicted values
print("Prediction for test set: {}".format(y_pred_mlr))

Prediction for test set: [ 1.2476011   0.65577429  0.43655391  0.95447531  1.6916062   0.05303671
  1.79233563  0.10352553  0.96634912  1.50711652  0.63936209  0.63936209
  1.20097629  1.90416492  0.63936209  1.99631754  0.75176779  0.77533341
  0.12508142  1.11956738  0.77082642  0.01782501  1.68009413  1.44049436
  0.84949526  1.84769165  1.31726475  0.43536572  1.43155911  0.82766848
  1.77329902  1.80722135  1.67643161  1.58519537  1.87016894  1.80722135
  1.47404128  0.63936209  0.03224099  1.87997218  1.97513936  0.75176779
  1.7344778   0.59797524  1.63257079  0.12889005  1.59516563  1.69052182
  1.54888983  1.72313567  1.68499119  0.82766848  0.63936209  1.9922892
  1.59716464  1.61420925  1.76231042  1.02634471  0.63936209  1.59488154
  1.11956738  1.2098244   1.16029123  1.77807706  0.3019415   1.74749137
  0.63936209  1.94058403  1.828859    1.94770692  1.70969642  1.7124275
  1.15977141  0.58277603  0.63936209  0.88742668  0.63936209 -0.21348399
  1.87212674  0.12394667  1.

In [ ]:
mlr_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred_mlr})
mlr_diff.head(100)

,Actual value,Predicted value
1247,0.0,1.247601
489,0.0,0.655774
1161,0.0,0.436554
2121,5.0,0.954475
1780,0.0,1.691606
...,...,...
134,0.0,0.647777
918,0.0,1.107420
65,1.0,0.837583
2449,1.0,0.305594


In [ ]:
meanAbErr = metrics.mean_absolute_error(y_test, y_pred_mlr)
meanSqErr = metrics.mean_squared_error(y_test, y_pred_mlr)
rootMeanSqErr = np.sqrt(metrics.mean_squared_error(y_test, y_pred_mlr))
print('R squared: {:.2f}'.format(mlr.score(X,Y)))
print('Mean Absolute Error:', meanAbErr)
print('Mean Square Error:', meanSqErr)
print('Root Mean Square Error:', rootMeanSqErr)

R squared: 0.20
Mean Absolute Error: 0.9960499024969358
Mean Square Error: 1.8760727890358662
Root Mean Square Error: 1.3696980649164494
